# Introduction

This notebook will train a very simple model that will compare two numbers and tell us if one is larger than the other.

We'll convert the model into something that can be used by tflite and then run it on the ESP32

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from tensorflow.data import Dataset
import numpy as np
import itertools

2023-07-16 10:01:50.054601: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-16 10:01:50.221030: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-16 10:01:50.221998: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-16 10:01:51.645213: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Create a dataset to train our model
We'll create a python generator and feed that through a tensorflow Dataset to train our model

In [63]:
def data_generator():
    while(True):
        number1 = np.random.uniform();
        number2 = np.random.uniform();
        # our input data is an array containing 2 numbers
        X = [number1, number2]
        # our label is 1 or 0
        Y = 1 if number2 > number1 else 0
        # our generator should return the input data and the label
        yield X, [Y]
        
# create a dataset from our generator
train_dataset = tf.data.Dataset.from_generator(
    data_generator, 
    output_types = (tf.float32, tf.int32),
    output_shapes=((2), (1))
)

train_dataset = train_dataset.batch(batch_size=30)
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))>

In [64]:
# Print the train_dataset
print(train_dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))>


## Our very simple mode

We don't need a very complicated model for our problem, so we'll just define a small neural network with an input layer and an output layer.

It's important that the activation function for the output should be sigmoid. This activation function will output a value between 0 and 1.

In [3]:
model = Sequential([
    Input(shape=(2)),
    Dense(5, activation='relu'),
    Dense(1, activation='sigmoid')
])

## Compile our model¶
For our loss function we need to use BinaryCrossentropy.

Crossentropy quantifies the difference between two probability distribution.

We have a binary distribution (True or False) so we use binary crossentropy to compare the output from our model with the true distribution.


In [68]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 15        
                                                                 
 dense_1 (Dense)             (None, 1)                 6         
                                                                 


Total params: 21 (84.00 Byte)
Trainable params: 21 (84.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [71]:
model.fit(
    train_dataset,
    steps_per_epoch=10,
    epochs=4
)

Epoch 1/4


10/10 [==============================] - 0s 9ms/step - loss: 0.0623 - accuracy: 0.9967
Epoch 2/4
10/10 [==============================] - 0s 7ms/step - loss: 0.0583 - accuracy: 1.0000
Epoch 3/4
10/10 [==============================] - 0s 11ms/step - loss: 0.0527 - accuracy: 1.0000
Epoch 4/4
10/10 [==============================] - 0s 5ms/step - loss: 0.0720 - accuracy: 0.9933


## Testing our model
We can feed in some values and see what our model predicts

In [73]:
test_X = np.array([
    [0.1, 0.2],
    [0.3, 0.4],
    [0.5, 0.1],
    [0.7, 0.2]
])
Y = model.predict_on_batch(test_X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print(Y)

[[0.91]
 [0.91]
 [0.00]
 [0.00]]


## Export our model for tflite
We need to convert our model into a tflite model

In [77]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_gen():
    for _ in range(10000):
        yield [
            np.array(
                [np.random.uniform(), np.random.uniform()]
            , dtype=np.float32)
        ]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_quant_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmppjnmdtaz/assets


INFO:tensorflow:Assets written to: /tmp/tmppjnmdtaz/assets
/home/milad/milad/1/TEZ/3esp32/2tflight/2tensorflow-lite-esp32/venv/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2023-07-16 11:11:00.303140: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-07-16 11:11:00.303166: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-07-16 11:11:00.303367: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmppjnmdtaz
2023-07-16 11:11:00.304033: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-07-16 11:11:00.304048: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmppjnmdtaz
2023-07-16 11:11:00.305932: I tensorflow/cc/saved_model/loader.cc:231] Rest

2344

## To convert to C++
We can then run this command to convert the model to c code.
```
xxd -i converted_model.tflite > model_data.cc
```

In [78]:
!xxd -i converted_model.tflite > firmware/src/model_data1.cc